In [1]:
import dask
import dask.dataframe as df
import numpy as np
import pandas as pd
from scipy import sparse as sp
from math import log
from collections import Counter
import functools 
import sklearn.metrics as metrics
np.random.seed(0)

In [2]:
pandas_df = pd.DataFrame(data={'col1':[1,2,3,3,2,1], 'col2':[4,5,4,4,5,5], 'col3':[3,4,3,4,3,4],
                                            'col4':[1,2,3,4,5,6], 'col5':[3,3,3,3,3,3], 'cols':['a','b','c','d','e','f']})

In [3]:
dask_df = df.from_pandas(pandas_df,npartitions=2)

In [4]:
dask_df.col1.unique()

Dask Series Structure:
npartitions=1
    int64
      ...
Name: col1, dtype: int64
Dask Name: unique-agg, 7 tasks